In [79]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD, NMF,KNNBasic,evaluate

import warnings; warnings.simplefilter('ignore')

In [145]:
movies = pd.read_csv('mldataset/movies_metadata.csv', low_memory=False)
movies

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,FALSE,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,10/30/1995,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,FALSE,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,12/15/1995,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,FALSE,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,12/22/1995,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,FALSE,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,12/22/1995,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,FALSE,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,2/10/1995,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
5,FALSE,NaN,60000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",...,12/15/1995,187436818.0,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0
6,FALSE,NaN,58000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",NaN,11860,tt0114319,en,Sabrina,An ugly duckling having undergone a remarkable...,...,12/15/1995,0.0,127.0,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,You are cordially invited to the most surprisi...,Sabrina,False,6.2,141.0
7,FALSE,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,45325,tt0112302,en,Tom and Huck,"A mischievous young boy, Tom Sawyer, witnesses...",...,12/22/1995,0.0,97.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The Original Bad Boys.,Tom and Huck,False,5.4,45.0
8,FALSE,NaN,35000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,9091,tt0114576,en,Sudden Death,International action superstar Jean Claude Van...,...,12/22/1995,64350171.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Terror goes into overtime.,Sudden Death,False,5.5,174.0
9,FALSE,"{'id': 645, 'name': 'James Bond Collection', '...",58000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",http://www.mgm.com/view/movie/757/Goldeneye/,710,tt0113189,en,GoldenEye,James Bond must unmask the mysterious head of ...,...,11/16/1995,352194034.0,130.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,No limits. No fears. No substitutes.,GoldenEye,False,6.6,1194.0


In [39]:
movies['genres'] = movies['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [40]:
movie_links = pd.read_csv('mldataset/links_small.csv')
movie_links = movie_links[movie_links['tmdbId'].notnull()]['tmdbId'].astype('int')

In [41]:
movies = movies.drop([19730, 29503, 35587])
#Check EDA Notebook for how and why I got these indices.
movies['id'] = movies['id'].astype('int')


In [42]:
#small_metadata['tagline'] = small_metadata['tagline'].fillna('')
#small_metadata['description'] = small_metadata['overview'] + small_metadata['tagline']
#small_metadata['description'] = small_metadata['description'].fillna('')

In [43]:
#small_metadata = small_metadata.reset_index()
#titles = small_metadata['title']
#indices = pd.Series(small_metadata.index, index=small_metadata['title'])

In [44]:
credits = pd.read_csv('mldataset/credits.csv')
keywords = pd.read_csv('mldataset/keywords.csv')

In [45]:
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')


In [46]:
movies = movies.merge(credits, on='id')
movies = movies.merge(keywords, on='id')

In [47]:
smd = movies[movies['id'].isin(movie_links)]
smd.shape

(9219, 27)

In [48]:
smd['cast'] = smd['cast'].apply(literal_eval)
smd['crew'] = smd['crew'].apply(literal_eval)
smd['keywords'] = smd['keywords'].apply(literal_eval)
smd['cast_size'] = smd['cast'].apply(lambda x: len(x))
smd['crew_size'] = smd['crew'].apply(lambda x: len(x))

In [49]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [50]:
smd['director'] = smd['crew'].apply(get_director)

In [51]:
smd['cast'] = smd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)

In [52]:
smd['keywords'] = smd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [53]:
smd['cast'] = smd['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [54]:
smd['director'] = smd['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
smd['director'] = smd['director'].apply(lambda x: [x,x, x])

In [55]:
s = smd.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'

In [56]:
s = s.value_counts()
s[:5]

independent film        610
woman director          550
murder                  399
duringcreditsstinger    327
based on novel          318
Name: keyword, dtype: int64

In [57]:
s = s[s > 1]

In [58]:
stemmer = SnowballStemmer('english')
stemmer.stem('dogs')

'dog'

In [59]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [60]:
smd['keywords'] = smd['keywords'].apply(filter_keywords)
smd['keywords'] = smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
smd['keywords'] = smd['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [61]:
smd['soup'] = smd['keywords'] + smd['director'] + smd['genres']
smd['soup'] = smd['soup'].apply(lambda x: ' '.join(x))

In [62]:
smd

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,video,vote_average,vote_count,cast,crew,keywords,cast_size,crew_size,director,soup
0,FALSE,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,False,7.7,5415.0,"[tomhanks, timallen, donrickles]","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[jealousi, toy, boy, friendship, friend, rival...",13,106,"[johnlasseter, johnlasseter, johnlasseter]",jealousi toy boy friendship friend rivalri boy...
1,FALSE,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,False,6.9,2413.0,"[robinwilliams, jonathanhyde, kirstendunst]","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[boardgam, disappear, basedonchildren'sbook, n...",26,16,"[joejohnston, joejohnston, joejohnston]",boardgam disappear basedonchildren'sbook newho...
2,FALSE,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,False,6.5,92.0,"[waltermatthau, jacklemmon, ann-margret]","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[fish, bestfriend, duringcreditssting]",7,4,"[howarddeutch, howarddeutch, howarddeutch]",fish bestfriend duringcreditssting howarddeutc...
3,FALSE,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,False,6.1,34.0,"[whitneyhouston, angelabassett, lorettadevine]","[{'credit_id': '52fe44779251416c91011acb', 'de...","[basedonnovel, interracialrelationship, single...",10,10,"[forestwhitaker, forestwhitaker, forestwhitaker]",basedonnovel interracialrelationship singlemot...
4,FALSE,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,False,5.7,173.0,"[stevemartin, dianekeaton, martinshort]","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[babi, midlifecrisi, confid, age, daughter, mo...",12,7,"[charlesshyer, charlesshyer, charlesshyer]",babi midlifecrisi confid age daughter motherda...
5,FALSE,NaN,60000000,"[Action, Crime, Drama, Thriller]",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",...,False,7.7,1886.0,"[alpacino, robertdeniro, valkilmer]","[{'credit_id': '52fe4292c3a36847f802916d', 'de...","[robberi, detect, bank, obsess, chase, shoot, ...",65,71,"[michaelmann, michaelmann, michaelmann]",robberi detect bank obsess chase shoot thief h...
6,FALSE,NaN,58000000,"[Comedy, Romance]",NaN,11860,tt0114319,en,Sabrina,An ugly duckling having undergone a remarkable...,...,False,6.2,141.0,"[harrisonford, juliaormond, gregkinnear]","[{'credit_id': '52fe44959251416c75039da9', 'de...","[pari, brotherbrotherrelationship, chauffeur, ...",57,53,"[sydneypollack, sydneypollack, sydneypollack]",pari brotherbrotherrelationship chauffeur long...
7,FALSE,NaN,0,"[Action, Adventure, Drama, Family]",NaN,45325,tt0112302,en,Tom and Huck,"A mischievous young boy, Tom Sawyer, witnesses...",...,False,5.4,45.0,"[jonathantaylorthomas, bradrenfro, rachaelleig...","[{'credit_id': '52fe46bdc3a36847f810f797', 'de...",[],7,4,"[peterhewitt, peterhewitt, peterhewitt]",peterhewitt peterhewitt peterhewitt Action Adv...
8,FALSE,NaN,35000000,"[Action, Adventure, Thriller]",NaN,9091,tt0114576,en,Sudden Death,International action superstar Jean Claude Van...,...,False,5.5,174.0,"[jean-claudevandamme, powersboothe, dorianhare...","[{'credit_id': '52fe44dbc3a36847f80ae0f1', 'de...","[terrorist, hostag, explos, vicepresid]",6,9,"[peterhyams, peterhyams, peterhyams]",terrorist hostag explos vicepresid peterhyams ...
9,FALSE,"{'id': 645, 'name': 'James Bond Collection', '...",58000000,"[Adventure, Action, Thriller]"

In [63]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
bag_of_word = count.fit_transform(smd['soup'])

In [64]:
bag_of_word

<9219x70899 sparse matrix of type '<class 'numpy.int64'>'
	with 184476 stored elements in Compressed Sparse Row format>

In [65]:
cosine_sim = cosine_similarity(bag_of_word, bag_of_word)

In [141]:
cosine_sim.shape

(9219, 9219)

In [67]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [68]:
reader = Reader()

In [69]:
ratings = pd.read_csv('mldataset/ratings_small.csv')

In [177]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
#data.split(n_folds=5)

In [178]:
svd = SVD()
evaluate(svd, data, measures=['RMSE'])


Evaluating RMSE of algorithm SVD.

------------
Fold 1
RMSE: 0.8963
------------
Fold 2
RMSE: 0.9030
------------
Fold 3
RMSE: 0.8882
------------
Fold 4
RMSE: 0.8970
------------
Fold 5
RMSE: 0.8974
------------
------------
Mean RMSE: 0.8964
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'rmse': [0.8962549838896235,
                             0.9030271563451314,
                             0.8881868509845073,
                             0.8969722277922315,
                             0.8974444441148994]})

In [76]:
nmf = NMF()
evaluate(nmf, data, measures=['RMSE'])

Evaluating RMSE of algorithm NMF.

------------
Fold 1
RMSE: 0.9408
------------
Fold 2
RMSE: 0.9468
------------
Fold 3
RMSE: 0.9558
------------
Fold 4
RMSE: 0.9503
------------
Fold 5
RMSE: 0.9475
------------
------------
Mean RMSE: 0.9482
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'rmse': [0.9407721090745125,
                             0.9467526946578111,
                             0.9558291041931941,
                             0.9502575450159287,
                             0.9474647086630944]})

In [86]:
trainset = data.build_full_trainset()
nmf.train(trainset)

In [87]:
nmf.predict(1, 302, 3)

Prediction(uid=1, iid=302, r_ui=3, est=2.7651284570453267, details={'was_impossible': False})

In [84]:
svd.predict(1, 302, 3)

Prediction(uid=1, iid=302, r_ui=3, est=2.8931794893409633, details={'was_impossible': False})

In [34]:
trainset = data.build_full_trainset()
svd.train(trainset)

In [95]:
knn = KNNBasic()
evaluate(knn, data, measures=['RMSE'])

Evaluating RMSE of algorithm KNNBasic.

------------
Fold 1
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9662
------------
Fold 2
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9674
------------
Fold 3
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9715
------------
Fold 4
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9696
------------
Fold 5
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9635
------------
------------
Mean RMSE: 0.9676
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'rmse': [0.9662184192378318,
                             0.9673817512722946,
                             0.9714912411210419,
                             0.9695824635396981,
                             0.9635483571187017]})

In [96]:
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

In [97]:
id_map = pd.read_csv('mldataset/links_small.csv')[['movieId', 'tmdbId']]
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
id_map.columns = ['movieId', 'id']
id_map = id_map.merge(smd[['title', 'id']], on='id').set_index('title')
#id_map = id_map.set_index('tmdbId')

In [98]:
indices_map = id_map.set_index('id')

In [157]:
def collaborative_filtering_recommendations(userId, title):
    idx = indices[title]
    tmdbId = id_map.loc[title]['id']
    #print(idx)
    #print(tmdbId)
    movie_id = id_map.loc[title]['movieId']
    #print(movie_id)
    
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    #print(sim_scores)
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    #print(sim_scores)
    movie_indices = [i[0] for i in sim_scores]
    #print(movie_indices)
    movies = smd.iloc[movie_indices][['title', 'id']]
    movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
    movies = movies.sort_values('est', ascending=False)
    return movies.head(10)

In [173]:
collaborative_filtering_recommendations(15, 'Star Wars')

,title,id,est
970,The Empire Strikes Back,1891,4.108698
6199,Star Wars: Episode III - Revenge of the Sith,1895,3.931241
2705,American Graffiti,838,3.432085
8865,Star Wars: The Force Awakens,140607,3.296132
983,Return of the Jedi,1892,3.115937
8079,Journey 2: The Mysterious Island,72545,3.056914
4137,Star Wars: Episode II - Attack of the Clones,1894,2.877047
7088,Star Wars: The Clone Wars,12180,2.237823
2120,Star Wars: Episode I - The Phantom Menace,1893,1.873009
4789,THX 1138,636,1.740039


In [169]:
knn = KNNBasic(k=10, min_k=5)


In [171]:
evaluate(svd, data, measures=['MAE'])


Evaluating MAE of algorithm SVD.

------------
Fold 1
MAE:  0.6880
------------
Fold 2
MAE:  0.6871
------------
Fold 3
MAE:  0.6963
------------
Fold 4
MAE:  0.6908
------------
Fold 5
MAE:  0.6862
------------
------------
Mean MAE : 0.6897
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'mae': [0.6879759839548474,
                             0.6871212707467347,
                             0.6963459273483696,
                             0.6907752399397824,
                             0.6861620293302547]})

In [172]:
evaluate(nmf, data, measures=['MAE'])

Evaluating MAE of algorithm NMF.

------------
Fold 1
MAE:  0.7261
------------
Fold 2
MAE:  0.7227
------------
Fold 3
MAE:  0.7336
------------
Fold 4
MAE:  0.7318
------------
Fold 5
MAE:  0.7237
------------
------------
Mean MAE : 0.7276
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'mae': [0.7261379042088961,
                             0.7226538686781716,
                             0.7335681580254326,
                             0.7318470669345681,
                             0.7236998790306701]})